# Integrantes GRUPO 4 -  GREAT EXPECTATIONS

- Carlos Almaraz Escobar
- Dilan Istael Condori Alejo
- Nelson Ernesto De La Quintanilla
- Paolo Ramos Mendez
- Percy Pedro Fuentes Ramos
- Gaston Nina Sossa

DATASET:

https://www.kaggle.com/datasets/carrie1/ecommerce-data

para correr el codigotuvimos que bajar la libreria a esta version especifica

great_expectations==0.17.23

La herramienta nos sirve para validar nuestra limpieza


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Imputer, StandardScaler, VectorAssembler
from pyspark.ml.stat import Correlation

# Inicializar Spark
spark = SparkSession.builder \
    .appName("DataQualityExample") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Cargar datos
df_spark = spark.read.option("header", "true").csv("data.csv", inferSchema=True)
df_spark.show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows



In [ ]:
# analisis de calidad

def spark_quality_analysis(df):
    """Análisis de calidad para Spark DataFrame"""
    print("=== ANÁLISIS DE CALIDAD SPARK ===")

    # Conteo de registros
    print(f"Total registros: {df.count()}")
    print(f"Total columnas: {len(df.columns)}")

    # Análisis de nulls por columna
    null_counts = df.select([
        count(when(isnan(c) | col(c).isNull(), c)).alias(c)
        for c in df.columns
    ]).collect()[0]

    print("\nVALORES NULOS POR COLUMNA:")
    for col_name, null_count in null_counts.asDict().items():
        if null_count > 0:
            percentage = (null_count / df.count()) * 100
            print(f"{col_name}: {null_count} ({percentage:.2f}%)")

    # Duplicados
    total_rows = df.count()
    unique_rows = df.distinct().count()
    duplicates = total_rows - unique_rows
    print(f"\nRegistros duplicados: {duplicates}")

    return df

spark_quality_analysis(df_spark)

=== ANÁLISIS DE CALIDAD SPARK ===
Total registros: 541909
Total columnas: 8

VALORES NULOS POR COLUMNA:
Description: 1454 (0.27%)
CustomerID: 135080 (24.93%)

Registros duplicados: 5268


DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [ ]:
# Limpieza Distribuida

def spark_data_cleaning(df):
    """Pipeline de limpieza para Spark"""

    # 1. Eliminar registros con muchos nulls
    threshold = 0.5  # Eliminar filas con >50% nulls
    min_non_null = int(threshold * len(df.columns))
    df_clean = df.dropna(thresh=min_non_null)

    # 2. Imputación por columna
    numeric_cols = [field.name for field in df_clean.schema.fields
                   if field.dataType in [IntegerType(), DoubleType(), FloatType()]]

    # Imputar con mediana
    for col_name in numeric_cols:
        median_val = df_clean.approxQuantile(col_name, [0.5], 0.01)[0]
        df_clean = df_clean.na.fill({col_name: median_val})

    # Imputar categóricas con moda
    string_cols = [field.name for field in df_clean.schema.fields
                  if field.dataType == StringType()]

    for col_name in string_cols:
        mode_val = df_clean.groupBy(col_name).count().orderBy(desc("count")).first()[0]
        df_clean = df_clean.na.fill({col_name: mode_val})

    # 3. Eliminar duplicados
    df_clean = df_clean.dropDuplicates()

    # 4. Detección de outliers usando IQR
    for col_name in numeric_cols:
        quantiles = df_clean.approxQuantile(col_name, [0.25, 0.75], 0.01)
        q1, q3 = quantiles[0], quantiles[1]
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr

        df_clean = df_clean.filter(
            (col(col_name) >= lower_bound) &
            (col(col_name) <= upper_bound)
        )

    return df_clean

df_spark_clean = spark_data_cleaning(df_spark)
print(f"Registros después de limpieza: {df_spark_clean.count()}")

In [ ]:
pip install great_expectations==0.17.23

In [ ]:
import pandas as pd
import numpy as np

# Cargar datos
df_clean = pd.read_csv("data.csv", encoding='latin-1')
df_clean.describe()

  return datetime.utcnow().replace(tzinfo=utc)



,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


  return datetime.utcnow().replace(tzinfo=utc)



In [ ]:
df_clean.info()

  return datetime.utcnow().replace(tzinfo=utc)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
# GREAT EXPECTATIONS

# Setup Great Expectations

import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import DataContextConfig

# Inicializar contexto
context = ge.get_context()

# Convertir pandas DataFrame a GE Dataset
ge_df = ge.from_pandas(df_clean)
ge_df

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp8731hza5' for ephemeral docs site


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


  return datetime.utcnow().replace(tzinfo=utc)



In [ ]:
# Definir Expectativas

def create_data_expectations(ge_df):
    """Define expectativas de calidad para el dataset"""

    # 1. Expectativas básicas de estructura
    ge_df.expect_table_row_count_to_be_between(min_value=1000, max_value=100000)
    ge_df.expect_table_column_count_to_equal(value=10)

    # 2. Expectativas de completitud
    ge_df.expect_column_values_to_not_be_null(column="StockCode")
    ge_df.expect_column_values_to_not_be_null(column="UnitPrice")

    # 3. Expectativas de valores únicos
    ge_df.expect_column_values_to_be_unique(column="StockCode")

    # 4. Expectativas de rango
    ge_df.expect_column_values_to_be_between(
        column="UnitPrice",
        min_value=0,
        max_value=10000
    )

    ge_df.expect_column_values_to_be_between(
        column="Quantity",
        min_value=0,
        max_value=1000
    )

    # 5. Expectativas de formato
    #StockCode
    #ge_df.expect_column_values_to_match_regex(
        #column="email",
        #regex=r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    #)

    # 6. Expectativas de categorías
#    ge_df.expect_column_values_to_be_in_set(
        #column="category",
        #value_set=["Electronics", "Clothing", "Books", "Home", "Sports"]
    #)

    # 7. Expectativas estadísticas
#    ge_df.expect_column_mean_to_be_between(
        #column="rating",
        #min_value=3.0,
        #max_value=4.5
    #)

    return ge_df

# Aplicar expectativas
ge_df_with_expectations = create_data_expectations(ge_df)

In [ ]:
# Validar y Generar Reporte

# Ejecutar validación
validation_result = ge_df_with_expectations.validate()

# Mostrar resultados
print("=== RESULTADOS DE VALIDACIÓN ===")
print(f"Validación exitosa: {validation_result.success}")
print(f"Total expectativas: {len(validation_result.results)}")

# Detalles de fallos
failed_expectations = [
    result for result in validation_result.results
    if not result.success
]

print(f"Expectativas fallidas: {len(failed_expectations)}")

for failed in failed_expectations:
    print(f"❌ {failed.expectation_config.expectation_type}")
    print(f"   Columna: {failed.expectation_config.kwargs.get('column', 'N/A')}")
    print(f"   Detalle: {failed.result}")

=== RESULTADOS DE VALIDACIÓN ===
Validación exitosa: False
Total expectativas: 7
Expectativas fallidas: 5
❌ expect_table_row_count_to_be_between
   Columna: N/A
   Detalle: {'observed_value': 541909}
❌ expect_table_column_count_to_equal
   Columna: N/A
   Detalle: {'observed_value': 8}
❌ expect_column_values_to_be_unique
   Columna: StockCode
   Detalle: {'element_count': 541909, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 541676, 'unexpected_percent': 99.95700385120011, 'unexpected_percent_total': 99.95700385120011, 'unexpected_percent_nonmissing': 99.95700385120011, 'partial_unexpected_list': ['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730', '22633', '22632', '84879', '22745', '22748', '22749', '22310', '84969', '22623', '22622', '21754', '21755', '21777']}
❌ expect_column_values_to_be_between
   Columna: UnitPrice
   Detalle: {'element_count': 541909, 'missing_count': 0, 'missing_percent': 0.0, 'unexpected_count': 12, 'unexpected_percent': 0.002

In [ ]:
# DATA QUALITY DASHBOARD

# Crear Visualizaciones de Calidad

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.offline as pyo

def create_quality_dashboard(df, validation_result):
    """Crea dashboard interactivo de calidad"""

    # 1. Completitud por columna
    missing_data = df.isnull().sum().reset_index()
    missing_data.columns = ['Column', 'Missing_Count']
    missing_data['Missing_Percentage'] = (missing_data['Missing_Count'] / len(df)) * 100

    fig1 = px.bar(missing_data, x='Column', y='Missing_Percentage',
                  title='Completitud por Columna (%)')
    fig1.update_layout(xaxis_tickangle=-45)

    # 2. Distribución de calidad por expectativas
    success_count = sum(1 for result in validation_result.results if result.success)
    fail_count = len(validation_result.results) - success_count

    fig2 = go.Figure(data=[
        go.Pie(labels=['Pasadas', 'Fallidas'],
               values=[success_count, fail_count],
               hole=0.3)
    ])
    fig2.update_layout(title='Distribución de Expectativas')

    # 3. Outliers por columna numérica
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    fig3 = make_subplots(rows=2, cols=2,
                        subplot_titles=[f'Outliers en {col}' for col in numeric_cols[:4]])

    for i, col in enumerate(numeric_cols[:4]):
        row = (i // 2) + 1
        col_pos = (i % 2) + 1

        fig3.add_trace(
            go.Box(y=df[col], name=col),
            row=row, col=col_pos
        )

    # 4. Correlación entre variables numéricas
    corr_matrix = df[numeric_cols].corr()
    fig4 = px.imshow(corr_matrix,
                     title='Matriz de Correlación',
                     color_continuous_scale='RdBu_r')

    # Mostrar gráficos
    fig1.show()
    fig2.show()
    fig3.show()
    fig4.show()

    return fig1, fig2, fig3, fig4

# Crear dashboard
dashboard_figs = create_quality_dashboard(df_clean, validation_result)


In [ ]:
# Reporte HTML Automatizado

def generate_html_report(df, validation_result, output_path="quality_report.html"):
    """Genera reporte HTML completo"""

    html_template = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Data Quality Report</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; }
            .header { background-color: #f0f0f0; padding: 20px; }
            .metric { display: inline-block; margin: 10px; padding: 15px;
                     background-color: #e8f4fd; border-radius: 5px; }
            .success { color: green; }
            .error { color: red; }
            table { border-collapse: collapse; width: 100%; }
            th, td { border: 1px solid #ddd; padding: 8px; text-align: left; }
            th { background-color: #f2f2f2; }
        </style>
    </head>
    <body>
        <div class="header">
            <h1>Data Quality Report</h1>
            <p>Generado: {timestamp}</p>
        </div>

        <h2>Métricas Generales</h2>
        <div class="metric">
            <strong>Total Registros:</strong> {total_rows}
        </div>
        <div class="metric">
            <strong>Total Columnas:</strong> {total_columns}
        </div>
        <div class="metric">
            <strong>Registros Duplicados:</strong> {duplicates}
        </div>
        <div class="metric">
            <strong>Expectativas Pasadas:</strong>
            <span class="success">{success_count}</span>
        </div>
        <div class="metric">
            <strong>Expectativas Fallidas:</strong>
            <span class="error">{fail_count}</span>
        </div>

        <h2>Completitud por Columna</h2>
        <table>
            <tr><th>Columna</th><th>Valores Faltantes</th><th>Porcentaje</th></tr>
            {completeness_table}
        </table>

        <h2>Expectativas Fallidas</h2>
        {failed_expectations_html}

    </body>
    </html>
    """

    # Calcular métricas
    total_rows = len(df)
    total_columns = len(df.columns)
    duplicates = df.duplicated().sum()
    success_count = sum(1 for result in validation_result.results if result.success)
    fail_count = len(validation_result.results) - success_count

    # Tabla de completitud
    missing_data = df.isnull().sum()
    completeness_rows = []
    for col, missing in missing_data.items():
        if missing > 0:
            percentage = (missing / total_rows) * 100
            completeness_rows.append(
                f"<tr><td>{col}</td><td>{missing}</td><td>{percentage:.2f}%</td></tr>"
            )

    completeness_table = "".join(completeness_rows)

    # Expectativas fallidas
    failed_expectations = [
        result for result in validation_result.results if not result.success
    ]

    failed_html = "<ul>"
    for failed in failed_expectations:
        failed_html += f"<li><strong>{failed.expectation_config.expectation_type}</strong>: "
        failed_html += f"{failed.result.get('partial_unexpected_list', 'Ver detalles')}</li>"
    failed_html += "</ul>"

    # Generar HTML
    html_content = html_template.format(
        timestamp=pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        total_rows=total_rows,
        total_columns=total_columns,
        duplicates=duplicates,
        success_count=success_count,
        fail_count=fail_count,
        completeness_table=completeness_table,
        failed_expectations_html=failed_html
    )

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Reporte generado: {output_path}")

# Generar reporte
generate_html_report(df_clean, validation_result)